In [10]:
import cv2
import numpy as np
import os
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [4]:
base_dir = "Images"

In [9]:
def extract_sift_features(image_dir):
    # Create the SIFT object
    sift = cv2.SIFT_create()

    # List will be used to hold the descriptors for all processed images
    descriptors_list = []

    # Iterate through each image in the input directory, covert to grayscale, and get the feature descriptors
    for img_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_file)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            continue
        _, descriptors = sift.detectAndCompute(image, None)
        if descriptors is not None:
            descriptors_list.append(descriptors)

    # Combine all descriptors into one array
    return np.vstack(descriptors_list) if descriptors_list else None

In [12]:
def detect_anomalies(image_dir, dbscan_model, scaler):
    sift = cv2.SIFT_create()
    anomalies = []

    for img_file in os.listdir(image_dir):
        img_path = os.path.join(image_dir, img_file)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            continue
        _, descriptors = sift.detectAndCompute(image, None)
        if descriptors is None:
            continue

        # Scale descriptors
        descriptors_scaled = scaler.transform(descriptors)
        labels = dbscan_model.fit_predict(descriptors_scaled)

        # Anomalous keypoints
        if -1 in labels:
            anomalies.append(img_file)

    return anomalies

In [14]:
def evaluate_anomaly_detection(image_path, ground_truth_path, anomaly_keypoints):
    # Load ground truth mask
    ground_truth_mask = cv2.imread(ground_truth_path, cv2.IMREAD_GRAYSCALE)
    ground_truth_mask = (ground_truth_mask > 0).astype(np.uint8)  # Binarize the mask

    # Create an anomaly map based on keypoints
    anomaly_map = np.zeros_like(ground_truth_mask)
    for kp in anomaly_keypoints:
        x, y = int(kp.pt[0]), int(kp.pt[1])
        cv2.circle(anomaly_map, (x, y), radius=5, color=1, thickness=-1)  # Mark anomalies

    # Compute evaluation metrics
    intersection = np.logical_and(ground_truth_mask, anomaly_map)
    union = np.logical_or(ground_truth_mask, anomaly_map)
    iou = np.sum(intersection) / np.sum(union) if np.sum(union) > 0 else 0

    return {"IoU": iou}

In [17]:
# First approach is to use SIFT feature extraction and the DBSCAN clustering algorithm to detect anomalies. Will use the
# images in the "train/good" directory to establish a baseline of features for defect-free images.

items = [f for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
# Iterate through the directories of images
for item in items:
    item_path = os.path.join(base_dir, item)
    train_path = os.path.join(item_path, 'train/good')
    test_path = os.path.join(item_path, 'test')
    ground_truth_path = os.path.join(item_path, 'ground_truth')

    print(f"Processing: {item}")

    # Extract features from defect-free training images
    train_features = extract_sift_features(train_path)
    print("Extracted SIFT features from:", item)

    # Scale features before giving to clustering algorithm
    scaler = StandardScaler()
    train_features_scaled = scaler.fit_transform(train_features)
    
    # Fit DBSCAN
    dbscan = DBSCAN(eps=0.5, min_samples=5, metric='euclidean')
    dbscan.fit(train_features_scaled)
    print("Fit DBSCAN to scaled train image features")

    # Detect anomalies in test images. Test imageset also contains a "good" subdirectory, which can be used for detecting false-positives.
    for defect_type in os.listdir(test_path):
        defect_path = os.path.join(test_path, defect_type)

        if defect_type == 'good':
            print("Processing 'good' images for false positives.")
            anomalies = detect_anomalies(defect_path, dbscan, scaler)
            print(f"False Positives in 'good' images: {anomalies}\n")
        else:
            # Process defect directories
            print(f"Processing defect type: {defect_type}")
            anomalies = detect_anomalies(defect_path, dbscan, scaler)
            print(f"Detected anomalies: {anomalies}\n")

            # Evaluate detected anomalies against ground truth masks
            ground_truth_defect_path = os.path.join(ground_truth_path, defect_type)
            for anomaly in anomalies:
                test_image_path = os.path.join(defect_path, anomaly)
                
                base_name, ext = os.path.splitext(anomaly)
                gt_image_name = f"{base_name}_mask{ext}"
                gt_image_path = os.path.join(ground_truth_defect_path, gt_image_name)

                if os.path.exists(gt_image_path):
                    results = evaluate_anomaly_detection(test_image_path, gt_image_path, anomaly_keypoints=[])
                    print(f"Evaluation for {anomaly}: {results}")
    break

Processing: capsule
Extracted SIFT features from: capsule
Fit DBSCAN to scaled train image features
Processing defect type: crack
Detected anomalies: ['000.png', '019.png', '008.png', '013.png', '017.png', '002.png', '021.png', '018.png', '007.png', '010.png', '005.png', '022.png', '004.png', '009.png', '003.png', '020.png', '011.png', '015.png', '016.png', '012.png', '006.png', '001.png', '014.png']

Evaluation for 000.png: {'IoU': 0.0}
Evaluation for 019.png: {'IoU': 0.0}
Evaluation for 008.png: {'IoU': 0.0}
Evaluation for 013.png: {'IoU': 0.0}
Evaluation for 017.png: {'IoU': 0.0}
Evaluation for 002.png: {'IoU': 0.0}
Evaluation for 021.png: {'IoU': 0.0}
Evaluation for 018.png: {'IoU': 0.0}
Evaluation for 007.png: {'IoU': 0.0}
Evaluation for 010.png: {'IoU': 0.0}
Evaluation for 005.png: {'IoU': 0.0}
Evaluation for 022.png: {'IoU': 0.0}
Evaluation for 004.png: {'IoU': 0.0}
Evaluation for 009.png: {'IoU': 0.0}
Evaluation for 003.png: {'IoU': 0.0}
Evaluation for 020.png: {'IoU': 0.0}
Eva